delint notebooks?


In [ ]:
var CLIEngine = require("eslint").CLIEngine;

var esLintConfig = {
    parser: "babel-eslint",
    "plugins": ["prettier"],
    envs: ["es6", "node", "browser", "mocha"],
    useEslintrc: false,
    extends: ["eslint:recommended", "google"],
    fix: true,
    'fix-dry-run': true,
    'fix-type': 'problem,suggestion,layout',
    rules: {
        'semi': 2,
        'prefer-const': 2,
        'no-var': 2,
        'no-undef': 2,
        'no-unused-vars': [2, {vars: 'all', args: 'all'}],
        'max-len': 2,
        'max-depth': 2,
        "prettier/prettier": "error",
        'quotes': [2, 'single']
    }
};

var cli = new CLIEngine(Object.assign(esLintConfig, {fix: false}));
var fix = new CLIEngine(Object.assign(esLintConfig, {fix: true}));

function delintCode(code) {
    // TODO: accept a file path
    if(typeof code === 'string') {
        code = [code]
    }
    try {
        return code
            .map(c => Object.assign(cli.executeOnText(c).results[0], {
                source: c,
                fixed: fix.executeOnText(c).results[0].output || c
            }))
    } catch (e) {
        console.log(e.message)
        console.log('Error: doing nothing to code.')
        return [{
            code: code
        }]
    }
}

module.exports = delintCode;




lint notebook?

google AI suggestion



In [ ]:
const { ESLint } = require('eslint');
const { defineConfig } = require('eslint/config');
const { FlatCompat } = require('@eslint/eslintrc');

// Translate ESLintRC-style configs into flat configs.
const compat = new FlatCompat({
  baseDirectory: __dirname,
  //recommendedConfig: typeScriptEsLintPlugin.configs['recommended'],
});

async function lintCode(code) {
  if (typeof code === 'string') {
    code = [code]
  }
  const eslint = new ESLint({
    allowInlineConfig: true, // Don't use .eslintrc config files
    //baseConfig: defineConfig([{}]),
    baseConfig: compat.config({
      parserOptions: {
        ecmaVersion: 2021,
        sourceType: 'module',
      },
      plugins: [],
      rules: {
        'no-unused-vars': 'warn',
        'no-console': 'off',
      },
    }),
    fix: false,
  });

  const results = []
  for (let i = 0; i < code.length; i++) {
    results.push(await eslint.lintText(code[i]));
  }
  return results.flat(1);
}

module.exports = lintCode


delint specific cells?

In [ ]:
var importer = require('../Core');
var delintCode = importer.import('lint notebooks');

async function delintCell(search) {
    let searchResults = importer.interpret(search);
    let result = await delintCode(searchResults.map(r => r.code))
    result = result.map((result, i) => result.messages.map(message => ({line: message.line + ':' + message.column, filename: searchResults[i].id, message: message.message}))).flat(1)
    if(typeof searchResults[0] !== 'undefined') {
        return result
    }
    return typeof search === 'string' && searchResults.length === 1
        ? result[0]
        : result;
}

module.exports = delintCell;


notebook lint watcher?

In [ ]:
var chokidar = require("chokidar");
var importer = require('../Core');
var delintCell = importer.import('delint specific cells');
var { glob } = importer.import('glob files')

// TODO: code analysis to combine blocks into modules?

var done = true;
function lintWatcher(files = '**/*.ipynb') {
  files = typeof files === 'string' ? [files] : files;
  console.log('watching ' + files + ' - ' + path.resolve('.'))
  let globFiles = glob(files, path.resolve('.'))
  var watcher = chokidar.watch(globFiles, {
    interval: 1000,
    atomic: 1000,
    awaitWriteFinish: true
  });
  watcher.on("change", function (filepath, event) {
    if (!done) {
      return;
    }
    console.log('running all delints on:', filepath);
    done = false;
    return delintCell(path.basename(filepath)).then(delint => {
      console.log('[' + path.basename(filepath) + ']', delint)
      done = true
    })
  });

  var stdin = process.openStdin();
  stdin.addListener("data", function (d) {
    stdin.close();
  });
}

module.exports = lintWatcher;



test de-linting service?


In [ ]:
var fs = require('fs');
var importer = require('../Core');
var delint = importer.import('delint notebooks');
var exportAndDeploy = importer.import('export deploy notebook');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = path.resolve(path.join(__dirname, '../Utilities/.modules'));

// just do it in load mode
if(typeof describe === 'undefined') {
    var it = ((l, func) => func());
    it.timeout = () => {};
    var beforeEach = ((func) => func());
    var describe = ((l, func) => func());
//    var describe = (() => {});
}

// TODO: make this a test script for exporting all notebooks on file change watcher
describe('de-linting service', () => {
    it('should de-lint a file', () => {
        const r = importer.interpret('diff code blocks');
        assert(r.filename.indexOf('diff.ipynb') > -1);
        return exportAndDeploy(['delint notebooks'])
            .then(() => exportNotebook(__filename))
            .then(() => delint(project))
            .then(r => {
                //console.log(r);
                //var filename = path.join(project, 'Utilities/diff/cell-' + cell.to + '.js');
                //return diffTwoTexts(cell.code, fs.readFileSync(filename).toString());
            })
            .catch(r => console.log(r))
    }).timeout(30000);
})



delint using webstorm?


In [ ]:
// TODO: display a tree of require dependencies and make sure it all still compiles properly
var path = require('path');
var {ncp} = require('ncp');
var importer = require('../Core');
var execCmd = importer.import('spawn child process');
ncp.limit = 16;

var webstormWin = 'C:\\Program Files\\JetBrains\\WebStorm 2017.1.1\\bin\\format.bat';
var webstormMac = '/Applications/WebStorm.app/Contents/bin/format.sh';
var webstorm = process.platform === 'win32' ? webstormWin : webstormMac;
var idea = path.resolve(path.join(__dirname, '../.idea'));

function delint(project) {
    // output .idea to set the framework preferences?
    // I think I overwrote the default, some way to env variables for this?
    return util.promisify(ncp)(idea, project + '/.idea')
        .then(() => execCmd(`"${webstorm}" "${project}"`, {cwd: project}))
}

module.exports = delint;

